In [2]:
from scipy.io import loadmat
import numpy as np 
import matplotlib.pyplot as plt 
from time import time 
import networkx as nx
from sklearn import preprocessing
from sklearn.cluster import KMeans
from scipy.sparse.linalg import eigs
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import normalized_mutual_info_score as nmi

import pickle
def save_variable(variable,filename):
  pickle.dump(variable,open(filename, "wb"))
def load_variable(filename):
  return pickle.load(open(filename,'rb')) 

In [3]:
# AO
# c to theta
def get_theta_from_c(lambda_,c,elogD):
  N = len(c)
  theta = np.zeros((N,N))
  for i in range(N):
    for j in range(i,N):
      theta[i,j] = np.random.laplace(loc=0.0, scale=1/(lambda_*(1-c[i]-c[j]+elogD[i,j])), size=1)
      theta[j,i] = theta[i,j]
  return theta

# theta to X
def get_X_from_theta(theta,N,D):
  X = np.zeros((N,D))
  for i in range(D):
    X[:,i] = np.random.multivariate_normal(mean = np.zeros((N,)), cov = np.linalg.inv(theta))
  return X

In [ ]:
N = 60
D = 30
c = np.zeros((60,1))
X= np.zeros((N,D,20))
elogD = np.zeros((N,N,20))
theta = np.zeros((N,N,20))
C = np.zeros((N,20))
c[0:6,0]=np.random.uniform(0.9,1,np.size(c[0:6,0]))
c[6:,0]=np.random.uniform(0,0.01,np.size(c[6:],0))
np.sort(c[:,0])[::-1]

In [ ]:
p=54
for k in range(20):
  C[0:p,k]=np.random.uniform(0.9,1,np.size(C[0:p,0]))
  C[p:,k]=np.random.uniform(0,0.01,np.size(C[p:,0]))
  C[:,k] = np.sort(C[:,k])[::-1]

  for i in range(N):
    for j in range(i,N):
      if (i<=p & j<=p):
        elogD[i,j,k] = np.random.uniform(1,1.05)
        elogD[j,i,k] = elogD[i,j,k]
      else:
        elogD[i,j,k] = np.random.uniform(1.2,1.25)
        elogD[j,i,k] = elogD[i,j,k]

  lambda_ = 0.1
  theta_aux = get_theta_from_c(lambda_,C[:,k],elogD[:,:,k])
  theta[:,:,k] = theta_aux + 300*np.eye(N)
  val, vec = eigs(theta[:,:,k],k=1,which='SM')
  X[:,:,k] = get_X_from_theta(theta[:,:,k],N,D)

In [8]:
# c to theta
# same as before
# c to X - GA-Nonlinear
def get_X_from_c_GA_Nonlinear(c,N,D,elogD):
  Z = np.ones((N,N))
  X = np.zeros((N,D))
  for i in range(N):
    for j in range(i,N):
      Z[i,j] = -c[i]-c[j]+elogD[i,j]
      Z[j,i] = Z[i,j]
  Z = Z+25*np.eye(N)
  for i in range(D):
    X[:,i] = np.random.multivariate_normal(mean = np.zeros((N,)), cov = np.linalg.inv(Z))
  return X,Z
for k in range(20):
  X[:,:,k],Z = get_X_from_c_GA_Nonlinear(C[:,k],N,D,elogD[:,:,k])  

In [12]:
# c to theta
# same as before
# c to X - GA-Affine-Bool

def get_X_from_c_GA_Affine_Bool(c,N,D,elogD):
  W = np.random.normal(0,1,(D,1))
  X = np.zeros((N,D))
  Q = np.zeros((N,D))
  for i in range(N):
    for j in range(D):
      Q[i,j] = 1/(1 + np.exp(-c[i]*W[j])) 
  X = np.random.binomial(1,Q)
  return X, W
for k in range(20):  
  X[:,:,k], W = get_X_from_c_GA_Affine_Bool(C[:,k],N,D,elogD[:,:,k])  

In [14]:
# extension
# c to theta
# same as before
# c to X - GA-Affine-Real

def get_X_from_c3(c,N,D,elogD):
  W = np.random.normal(0,1,(D,1))
  X = np.zeros((N,D))
  for i in range(N):
    for j in range(D):
      X[i,j] = np.random.normal(c[i]*W[j],1) 
  return X, W

for k in range(20):    
  X[:,:,k], W = get_X_from_c3(C[:,k],N,D,elogD[:,:,k])